In [1]:
import torch, torch.jit as jit
import torch.nn as nn
import torch.nn.functional as F
import os
os.environ["PYTORCH_JIT"] = "0"

In [2]:
from networks import *

In [3]:
gen_x = Generator().cuda()(n_samples=16)
print(f"Gen size:{gen_x.size()}")
print(f"Disc size:{Discriminator().cuda()(gen_x).size()}")

Gen size:torch.Size([16, 3, 64, 64])
Disc size:torch.Size([16, 1])


C:\ProgramData\Anaconda\Anaconda3\envs\ptml\lib\site-packages\torch\nn\functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


### Train

In [4]:
from train import *
from q1_3 import *
from networks import Discriminator, Generator
from train import train_model

In [7]:
!git remote -v

origin	https://github.com/visual-learning/generative-modeling (fetch)
origin	https://github.com/visual-learning/generative-modeling (push)


In [5]:
gen = Generator().cuda().to(memory_format=torch.channels_last)
disc = Discriminator().cuda().to(memory_format=torch.channels_last)
prefix = "data_gan/"
os.makedirs(prefix, exist_ok=True)

# TODO 1.3.2: Run this line of code.
train_model(
    gen,
    disc,
    num_iterations=int(3e4),
    batch_size=256,
    prefix=prefix,
    gen_loss_fn=compute_generator_loss,
    disc_loss_fn=compute_discriminator_loss,
    log_period=1000,
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: 3m-m. Use `wandb login --relogin` to force relogin


RuntimeError: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript (most recent call last):
  File "C:\Users\Mo\OneDrive\Notes\CMU\16824 - Visual Learning\github\generative-modeling\gan\networks.py", line 325, in forward
        # TODO 1.1: Forward the discriminator assuming a batch of images have been passed in.
        # Make sure to flatten the output of the convolutional layers and sum across the image dimensions before passing to the output layer!
        x = self.layers(x)   # (*, 128, 8, 8)
            ~~~~~~~~~~~ <--- HERE
        x = torch.sum(x.view(x.size(0), x.size(1),-1), dim=-1)
        x = self.dense_out(x.view(x.size(0), -1))
  File "C:\ProgramData\Anaconda\Anaconda3\envs\ptml\lib\site-packages\torch\nn\modules\container.py", line 141, in forward
    def forward(self, input):
        for module in self:
            input = module(input)
                    ~~~~~~ <--- HERE
        return input
  File "C:\Users\Mo\OneDrive\Notes\CMU\168